In [59]:
import pandas as pd

project_length_df = pd.read_csv(r'C:\Users\clint\Desktop\RA Task\project_length_extracted.csv')

In [60]:
project_length_df

,filename,project_lengths
0,170561.pdf,['10.15']
1,170626.pdf,[]
2,170634.pdf,['13.93']
3,174015.pdf,['5.45']
4,174016.pdf,['1.22']
...,...,...
206,180622.pdf,['24.61']
207,184002.pdf,['1.67']
208,187023.pdf,['17.59']
209,188000.pdf,['5.45']


In [61]:
empty_project_lengths_count = project_length_df['project_lengths'].apply(lambda x: x == '[]').sum()
print(f"Number of entries in project_lengths where the entry is only []: {empty_project_lengths_count}")

Number of entries in project_lengths where the entry is only []: 44


In [62]:
df_10 = pd.read_csv(r'C:\Users\clint\Desktop\RA Task\10.csv')
df_10

,state,county,fips,year,project_start,project_id,route,mileage,lanes,project_duration_days,eng_estimate_mils,win_bid_mils,cost_mils,num_bidders,bidders_list,all_routes,cost_overrun_pct,Project Num,bidders_count
0,Ohio,Paulding,39125,2018,2018-05-24 00:00:00,105522,111,12.982,2.0,99.0,0.943,0.957859,1.047510,2.0,"Shelly, Gerken Paving",NaN,9.359511,180326,2
1,Ohio,Wyandot,39175,2018,2018-11-15 00:00:00,88832,23,NaN,4.0,290.0,3.702,3.236775,3.304783,3.0,"Shelly, Shelly & Sands, Kokosing Construction","['23', '23', '23']",2.101101,180569,3
2,Ohio,Butler,39017,2018,2018-01-18 00:00:00,94263,73,NaN,2.0,195.0,0.287,0.258900,0.232678,3.0,"Barrett Paving Materials, John R Jurgensen, Ra...",['73'],-10.128362,180006,3
3,Ohio,Franklin,39049,2018,2018-01-18 00:00:00,76467,270 / 315,NaN,4.0,255.0,4.904,6.101481,6.991613,3.0,"Kokosing Construction, Shelly, Shelly & Sands","['270', '270', '270', '270', '270', '270', '27...",14.588788,180012,3
4,Ohio,Hocking,39073,2018,2018-01-18 00:00:00,101555,33,NaN,2.0,255.0,0.435,0.553756,0.531749,2.0,"Shelly & Sands, Shelly",['33'],-3.974212,180020,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Ohio,Harrison,39067,2018,2018-12-20 00:00:00,91844,250 / 9,NaN,2.0,284.0,2.249,2.284000,2.212447,3.0,"Nls Paving, Shelly & Sands, Barbicas Construction","['9', '250']",-3.132812,180609,3
199,Ohio,Highland,39071,2018,2018-12-20 00:00:00,84622,138 / 753,NaN,2.0,223.0,1.508,1.494436,1.658765,2.0,"Shelly, John R Jurgensen","['138', '138', '138', '753']",10.996011,180610,2
200,Ohio,Madison,39097,2018,2018-12-20 00:00:00,105547,42,NaN,2.0,284.0,3.727,3.611668,3.535001,4.0,"Shelly & Sands, Kokosing Construction, Shelly,...","['42', '42', '42']",-2.122762,180611,4
201,Ohio,Clinton,39027,2018,2018-12-20 00:00:00,87300,251 / 68 / 350,NaN,2.0,589.0,4.893,5.441520,5.331566,1.0,John R Jurgensen,"['68', '68', '68', '68', '68', '68', '251', '3...",-2.020640,180621,1


In [63]:
import ast

# Extract project numbers from filenames and create a mapping
project_length_mapping = {}

for index, row in project_length_df.iterrows():
    # Extract project number from filename (e.g., '170561.pdf' -> '170561')
    filename = row['filename']
    project_num = filename.replace('.pdf', '')  # Remove .pdf extension
    
    # Get project length - parse the string representation of list
    project_lengths_str = row['project_lengths']
    try:
        project_lengths = ast.literal_eval(project_lengths_str)
        if project_lengths and len(project_lengths) > 0:
            # Convert string to float
            project_length = float(project_lengths[0])  # Take the first value
            project_length_mapping[int(project_num)] = project_length
    except (ValueError, SyntaxError):
        # Skip if parsing fails
        continue

print(f"Created mapping for {len(project_length_mapping)} projects")
print("Sample mappings:")
for i, (proj_num, length) in enumerate(list(project_length_mapping.items())[:5]):
    print(f"Project {proj_num}: {length} miles")

Created mapping for 167 projects
Sample mappings:
Project 170561: 10.15 miles
Project 170634: 13.93 miles
Project 174015: 5.45 miles
Project 174016: 1.22 miles
Project 180003: 9.19 miles


In [64]:
# Now fill the mileage column in df_10 using the mapping
matches_found = 0
matches_not_found = 0

for index, row in df_10.iterrows():
    project_num = row['Project Num']
    
    if project_num in project_length_mapping:
        # Fill the mileage with the corresponding project length
        df_10.at[index, 'mileage'] = project_length_mapping[project_num]
        matches_found += 1
    else:
        matches_not_found += 1

print(f"Matches found: {matches_found}")
print(f"Matches not found: {matches_not_found}")
print(f"Total rows in df_10: {len(df_10)}")

# Check how many NaN values remain in mileage column
nan_count = df_10['mileage'].isna().sum()
print(f"Remaining NaN values in mileage column: {nan_count}")

# Show some examples of filled data
print("\nSample of filled data:")
filled_data = df_10[df_10['mileage'].notna()][['Project Num', 'mileage']].head()
print(filled_data)

Matches found: 163
Matches not found: 40
Total rows in df_10: 203
Remaining NaN values in mileage column: 40

Sample of filled data:
   Project Num  mileage
0       180326     6.49
1       180569     6.19
2       180006     0.50
4       180020     1.23
5       180024     8.14


In [65]:
# Display the updated df_10 with filled mileage data
print("\nUpdated df_10 with filled mileage data:")
df_10


Updated df_10 with filled mileage data:


,state,county,fips,year,project_start,project_id,route,mileage,lanes,project_duration_days,eng_estimate_mils,win_bid_mils,cost_mils,num_bidders,bidders_list,all_routes,cost_overrun_pct,Project Num,bidders_count
0,Ohio,Paulding,39125,2018,2018-05-24 00:00:00,105522,111,6.49,2.0,99.0,0.943,0.957859,1.047510,2.0,"Shelly, Gerken Paving",NaN,9.359511,180326,2
1,Ohio,Wyandot,39175,2018,2018-11-15 00:00:00,88832,23,6.19,4.0,290.0,3.702,3.236775,3.304783,3.0,"Shelly, Shelly & Sands, Kokosing Construction","['23', '23', '23']",2.101101,180569,3
2,Ohio,Butler,39017,2018,2018-01-18 00:00:00,94263,73,0.50,2.0,195.0,0.287,0.258900,0.232678,3.0,"Barrett Paving Materials, John R Jurgensen, Ra...",['73'],-10.128362,180006,3
3,Ohio,Franklin,39049,2018,2018-01-18 00:00:00,76467,270 / 315,NaN,4.0,255.0,4.904,6.101481,6.991613,3.0,"Kokosing Construction, Shelly, Shelly & Sands","['270', '270', '270', '270', '270', '270', '27...",14.588788,180012,3
4,Ohio,Hocking,39073,2018,2018-01-18 00:00:00,101555,33,1.23,2.0,255.0,0.435,0.553756,0.531749,2.0,"Shelly & Sands, Shelly",['33'],-3.974212,180020,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Ohio,Harrison,39067,2018,2018-12-20 00:00:00,91844,250 / 9,11.23,2.0,284.0,2.249,2.284000,2.212447,3.0,"Nls Paving, Shelly & Sands, Barbicas Construction","['9', '250']",-3.132812,180609,3
199,Ohio,Highland,39071,2018,2018-12-20 00:00:00,84622,138 / 753,11.69,2.0,223.0,1.508,1.494436,1.658765,2.0,"Shelly, John R Jurgensen","['138', '138', '138', '753']",10.996011,180610,2
200,Ohio,Madison,39097,2018,2018-12-20 00:00:00,105547,42,NaN,2.0,284.0,3.727,3.611668,3.535001,4.0,"Shelly & Sands, Kokosing Construction, Shelly,...","['42', '42', '42']",-2.122762,180611,4
201,Ohio,Clinton,39027,2018,2018-12-20 00:00:00,87300,251 / 68 / 350,13.06,2.0,589.0,4.893,5.441520,5.331566,1.0,John R Jurgensen,"['68', '68', '68', '68', '68', '68', '251', '3...",-2.020640,180621,1


In [66]:
# Summary of the operation
print("=== MILEAGE FILLING SUMMARY ===")
print(f"Total projects in df_10: {len(df_10)}")
print(f"Projects with mileage data filled: {matches_found}")
print(f"Projects without matching data: {matches_not_found}")
print(f"Success rate: {(matches_found/len(df_10)*100):.1f}%")

# Show some projects that couldn't be matched
print("\nSample projects that couldn't be matched:")
unmatched = df_10[df_10['mileage'].isna()][['Project Num']].head(5)
print(unmatched['Project Num'].tolist())

# Check which project numbers from project_length_df were actually used
used_project_nums = set(df_10[df_10['mileage'].notna()]['Project Num'].tolist())
all_project_nums = set(project_length_mapping.keys())
unused_project_nums = all_project_nums - used_project_nums
print(f"\nProject numbers in mapping but not used: {len(unused_project_nums)}")
if len(unused_project_nums) > 0:
    print(f"Sample unused: {list(unused_project_nums)[:5]}")

=== MILEAGE FILLING SUMMARY ===
Total projects in df_10: 203
Projects with mileage data filled: 163
Projects without matching data: 40
Success rate: 80.3%

Sample projects that couldn't be matched:
[180012, 180043, 180101, 180113, 180114]

Project numbers in mapping but not used: 5
Sample unused: [174016, 170561, 180004, 170634, 174015]


In [67]:
# Show before and after comparison for the original mileage column
print("\n=== BEFORE AND AFTER COMPARISON ===")
print("Original mileage (first 5 non-null values):")
original_mileage_sample = [12.982]  # The original value we saw
print(f"Row 0 had: 12.982 miles")
print(f"Now Row 0 has: {df_10.iloc[0]['mileage']} miles (from project_length_df)")

print("\nNote: The original 'mileage' column values have been replaced with")
print("values extracted from the PDF project length data based on Project Num matching.")


=== BEFORE AND AFTER COMPARISON ===
Original mileage (first 5 non-null values):
Row 0 had: 12.982 miles
Now Row 0 has: 6.49 miles (from project_length_df)

Note: The original 'mileage' column values have been replaced with
values extracted from the PDF project length data based on Project Num matching.


In [68]:
# DEBUGGING: Detailed analysis of missing matches
import os

print("=== DEBUGGING MISSING MATCHES ===")

# Get all unmatched project numbers
unmatched_projects = df_10[df_10['mileage'].isna()]['Project Num'].tolist()
print(f"Total unmatched projects: {len(unmatched_projects)}")
print(f"Unmatched project numbers: {sorted(unmatched_projects)}")

# Check if PDF files exist for unmatched projects
print("\n=== CHECKING PDF EXISTENCE ===")
pdf_folder = r'C:\Users\clint\Desktop\RA Task\Downloads'
missing_pdfs = []
existing_pdfs_but_no_data = []

for proj_num in unmatched_projects[:10]:  # Check first 10 for detailed analysis
    pdf_filename = f"{proj_num}.pdf"
    pdf_path = os.path.join(pdf_folder, pdf_filename)
    
    if os.path.exists(pdf_path):
        # PDF exists, check if it's in project_length_df
        pdf_in_df = project_length_df[project_length_df['filename'] == pdf_filename]
        if not pdf_in_df.empty:
            project_lengths_str = pdf_in_df.iloc[0]['project_lengths']
            print(f"Project {proj_num}: PDF exists, project_lengths = {project_lengths_str}")
            if project_lengths_str == '[]':
                existing_pdfs_but_no_data.append(proj_num)
        else:
            print(f"Project {proj_num}: PDF exists but NOT in project_length_df")
    else:
        print(f"Project {proj_num}: PDF file MISSING at {pdf_path}")
        missing_pdfs.append(proj_num)

print(f"\nSummary:")
print(f"PDFs missing from filesystem: {missing_pdfs}")
print(f"PDFs exist but have empty project_lengths []: {existing_pdfs_but_no_data}")

=== DEBUGGING MISSING MATCHES ===
Total unmatched projects: 40
Unmatched project numbers: [180012, 180043, 180101, 180113, 180114, 180115, 180116, 180140, 180153, 180172, 180174, 180189, 180222, 180223, 180249, 180272, 180285, 180291, 180294, 180312, 180318, 180319, 180331, 180401, 180420, 180421, 180422, 180448, 180477, 180491, 180508, 180510, 180535, 180542, 180549, 180555, 180570, 180574, 180601, 180611]

=== CHECKING PDF EXISTENCE ===
Project 180012: PDF exists, project_lengths = []
Project 180043: PDF exists, project_lengths = []
Project 180101: PDF exists, project_lengths = []
Project 180113: PDF exists, project_lengths = []
Project 180114: PDF exists, project_lengths = []
Project 180115: PDF exists, project_lengths = []
Project 180116: PDF exists, project_lengths = []
Project 180140: PDF exists, project_lengths = []
Project 180153: PDF exists, project_lengths = []
Project 180172: PDF exists, project_lengths = []

Summary:
PDFs missing from filesystem: []
PDFs exist but have empt

In [69]:
# Check what project numbers are available vs what's needed
print("\n=== AVAILABILITY ANALYSIS ===")

# Get all project numbers from df_10
all_df10_projects = set(df_10['Project Num'].tolist())
print(f"Total project numbers in df_10: {len(all_df10_projects)}")

# Get all project numbers from project_length_df (including empty ones)
all_available_projects = set()
for index, row in project_length_df.iterrows():
    filename = row['filename']
    project_num = int(filename.replace('.pdf', ''))
    all_available_projects.add(project_num)

print(f"Total project numbers in project_length_df: {len(all_available_projects)}")

# Get project numbers that have actual data (non-empty project_lengths)
available_with_data = set(project_length_mapping.keys())
print(f"Project numbers with actual length data: {len(available_with_data)}")

# Find completely missing projects (not even in project_length_df)
completely_missing = all_df10_projects - all_available_projects
print(f"\nProjects completely missing from project_length_df: {len(completely_missing)}")
if completely_missing:
    print(f"Missing projects: {sorted(list(completely_missing))}")

# Find projects with empty data
available_but_empty = all_available_projects & all_df10_projects - available_with_data
print(f"\nProjects in project_length_df but with empty data: {len(available_but_empty)}")
if available_but_empty:
    print(f"Empty data projects: {sorted(list(available_but_empty))}")


=== AVAILABILITY ANALYSIS ===
Total project numbers in df_10: 202
Total project numbers in project_length_df: 211
Project numbers with actual length data: 167

Projects completely missing from project_length_df: 0

Projects in project_length_df but with empty data: 40
Empty data projects: [180012, 180043, 180101, 180113, 180114, 180115, 180116, 180140, 180153, 180172, 180174, 180189, 180222, 180223, 180249, 180272, 180285, 180291, 180294, 180312, 180318, 180319, 180331, 180401, 180420, 180421, 180422, 180448, 180477, 180491, 180508, 180510, 180535, 180542, 180549, 180555, 180570, 180574, 180601, 180611]


In [70]:
# IMPROVED MATCHING: Handle empty entries as "NA"
print("\n=== IMPLEMENTING IMPROVED MATCHING ===")

# Create improved mapping that includes "NA" for empty entries
improved_mapping = {}

for index, row in project_length_df.iterrows():
    filename = row['filename']
    project_num = int(filename.replace('.pdf', ''))
    
    project_lengths_str = row['project_lengths']
    try:
        project_lengths = ast.literal_eval(project_lengths_str)
        if project_lengths and len(project_lengths) > 0:
            # Has actual data
            project_length = float(project_lengths[0])
            improved_mapping[project_num] = project_length
        else:
            # Empty list - mark as NA
            improved_mapping[project_num] = "NA"
    except (ValueError, SyntaxError):
        # Parsing failed - mark as NA
        improved_mapping[project_num] = "NA"

print(f"Improved mapping created with {len(improved_mapping)} entries")

# Count NA entries
na_count = sum(1 for v in improved_mapping.values() if v == "NA")
print(f"Entries marked as 'NA': {na_count}")
print(f"Entries with actual data: {len(improved_mapping) - na_count}")

# Apply improved matching
print("\nApplying improved matching...")
improved_matches_found = 0
improved_matches_not_found = 0
na_filled = 0

# Reset mileage column first
df_10['mileage'] = None

for index, row in df_10.iterrows():
    project_num = row['Project Num']
    
    if project_num in improved_mapping:
        value = improved_mapping[project_num]
        df_10.at[index, 'mileage'] = value
        improved_matches_found += 1
        if value == "NA":
            na_filled += 1
    else:
        improved_matches_not_found += 1

print(f"\nImproved Results:")
print(f"Matches found: {improved_matches_found}")
print(f"Matches not found: {improved_matches_not_found}")
print(f"Filled with 'NA': {na_filled}")
print(f"Filled with actual data: {improved_matches_found - na_filled}")
print(f"Success rate: {(improved_matches_found/len(df_10)*100):.1f}%")

# Show the projects that are still unmatched (completely missing)
still_unmatched = df_10[df_10['mileage'].isna()]
if not still_unmatched.empty:
    print(f"\nStill unmatched projects ({len(still_unmatched)}):")
    print(f"These are completely missing from project_length_df: {sorted(still_unmatched['Project Num'].tolist())}")
else:
    print("\n🎉 ALL PROJECTS MATCHED! No remaining unmatched projects.")


=== IMPLEMENTING IMPROVED MATCHING ===
Improved mapping created with 211 entries
Entries marked as 'NA': 44
Entries with actual data: 167

Applying improved matching...

Improved Results:
Matches found: 203
Matches not found: 0
Filled with 'NA': 40
Filled with actual data: 163
Success rate: 100.0%

🎉 ALL PROJECTS MATCHED! No remaining unmatched projects.


In [71]:
# Show sample results
print("\n=== SAMPLE RESULTS ===")

print("Sample of projects with numeric mileage data:")
numeric_data = df_10[df_10['mileage'].apply(lambda x: isinstance(x, (int, float)) and str(x) != 'nan')]
if not numeric_data.empty:
    print(numeric_data[['Project Num', 'mileage']].head())

print("\nSample of projects with 'NA' (empty data):")
na_data = df_10[df_10['mileage'] == "NA"]
if not na_data.empty:
    print(na_data[['Project Num', 'mileage']].head())

print("\nSample of completely unmatched projects:")
unmatched_data = df_10[df_10['mileage'].isna()]
if not unmatched_data.empty:
    print(unmatched_data[['Project Num', 'mileage']].head())

# Final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Total projects in df_10: {len(df_10)}")
print(f"✅ Matched with numeric data: {len(numeric_data)}")
print(f"⚠️  Matched with 'NA' (empty): {len(na_data)}")
print(f"❌ Completely unmatched: {len(unmatched_data)}")
print(f"📊 Overall success rate: {((len(numeric_data) + len(na_data))/len(df_10)*100):.1f}%")


=== SAMPLE RESULTS ===
Sample of projects with numeric mileage data:
   Project Num mileage
0       180326    6.49
1       180569    6.19
2       180006     0.5
4       180020    1.23
5       180024    8.14

Sample of projects with 'NA' (empty data):
    Project Num mileage
3        180012      NA
13       180043      NA
30       180101      NA
37       180113      NA
38       180114      NA

Sample of completely unmatched projects:

=== FINAL SUMMARY ===
Total projects in df_10: 203
✅ Matched with numeric data: 163
⚠️  Matched with 'NA' (empty): 40
❌ Completely unmatched: 0
📊 Overall success rate: 100.0%


In [72]:
df_10

,state,county,fips,year,project_start,project_id,route,mileage,lanes,project_duration_days,eng_estimate_mils,win_bid_mils,cost_mils,num_bidders,bidders_list,all_routes,cost_overrun_pct,Project Num,bidders_count
0,Ohio,Paulding,39125,2018,2018-05-24 00:00:00,105522,111,6.49,2.0,99.0,0.943,0.957859,1.047510,2.0,"Shelly, Gerken Paving",NaN,9.359511,180326,2
1,Ohio,Wyandot,39175,2018,2018-11-15 00:00:00,88832,23,6.19,4.0,290.0,3.702,3.236775,3.304783,3.0,"Shelly, Shelly & Sands, Kokosing Construction","['23', '23', '23']",2.101101,180569,3
2,Ohio,Butler,39017,2018,2018-01-18 00:00:00,94263,73,0.5,2.0,195.0,0.287,0.258900,0.232678,3.0,"Barrett Paving Materials, John R Jurgensen, Ra...",['73'],-10.128362,180006,3
3,Ohio,Franklin,39049,2018,2018-01-18 00:00:00,76467,270 / 315,NA,4.0,255.0,4.904,6.101481,6.991613,3.0,"Kokosing Construction, Shelly, Shelly & Sands","['270', '270', '270', '270', '270', '270', '27...",14.588788,180012,3
4,Ohio,Hocking,39073,2018,2018-01-18 00:00:00,101555,33,1.23,2.0,255.0,0.435,0.553756,0.531749,2.0,"Shelly & Sands, Shelly",['33'],-3.974212,180020,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Ohio,Harrison,39067,2018,2018-12-20 00:00:00,91844,250 / 9,11.23,2.0,284.0,2.249,2.284000,2.212447,3.0,"Nls Paving, Shelly & Sands, Barbicas Construction","['9', '250']",-3.132812,180609,3
199,Ohio,Highland,39071,2018,2018-12-20 00:00:00,84622,138 / 753,11.69,2.0,223.0,1.508,1.494436,1.658765,2.0,"Shelly, John R Jurgensen","['138', '138', '138', '753']",10.996011,180610,2
200,Ohio,Madison,39097,2018,2018-12-20 00:00:00,105547,42,NA,2.0,284.0,3.727,3.611668,3.535001,4.0,"Shelly & Sands, Kokosing Construction, Shelly,...","['42', '42', '42']",-2.122762,180611,4
201,Ohio,Clinton,39027,2018,2018-12-20 00:00:00,87300,251 / 68 / 350,13.06,2.0,589.0,4.893,5.441520,5.331566,1.0,John R Jurgensen,"['68', '68', '68', '68', '68', '68', '251', '3...",-2.020640,180621,1


In [73]:
df_10.to_csv('13.csv', index=False)